Libraries necessary to extract data from the wiki page

In [112]:
import pandas as pd 

import requests
from bs4 import BeautifulSoup

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Read the page and convert it to pandas' dataframe.

In [87]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'xml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]


Remove rows where borough is "Not assigned"

In [88]:
df = df[df['Borough']!='Not assigned']


Combine Neighborhood values if postcode and borough are the same. 

In [ ]:
df['Neighborhood'] = df.groupby(['Postcode','Borough'])['Neighborhood'].transform(lambda x: ','.join(x))
df = df.drop_duplicates()

In case when Neighborhood is "Not assigned" Make it the same as Borough.

In [89]:
df['Neighborhood'] = df.apply(
    lambda row: row['Borough'] if (row['Neighborhood']=='Not assigned') else row['Neighborhood'],
    axis=1
)


In [90]:
df.head(11)

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights,Lawrence Manor"
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,"Rouge,Malvern"
13,M3B,North York,Don Mills North
14,M4B,East York,"Woodbine Gardens,Parkview Hill"
16,M5B,Downtown Toronto,"Ryerson,Garden District"


In [91]:
df.shape

(103, 3)

In [95]:
df_geo = pd.read_csv('geospatial_data.csv')
df_geo


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [98]:
df_new = pd.merge(left=df,right=df_geo, left_on='Postcode', right_on='Postal Code')
df_new = df_new.drop('Postal Code', axis=1)


In [103]:
CLIENT_ID = 'TOM5H323NAKMWNAJILZ2NFHBQ2PHVXUQFVC5G4HL430Y4XTX' # your Foursquare ID
CLIENT_SECRET = 'UKTT0JDAWCEMN2H1POFFCH3T4GTWR31UFAX4FS4F53Q4LMSH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [109]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.653963, -79.387207.


In [114]:

# create map of Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [115]:
df_new

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
101,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout...",43.636258,-79.498509
